# Harvesting data from the web: APIs  

### A first API

[Chronicling America](http://chroniclingamerica.loc.gov/about/) is a joint project of the National Endowment for the Humanities and the Library of Congress .

Search for articles that mention "[lynching](http://chroniclingamerica.loc.gov/search/pages/results/?andtext=lynching)".

![](https://raw.githubusercontent.com/nealcaren/UiOBigData/master/notebooks/images/chron.png)

![](https://github.com/nealcaren/ScrapingData/raw/master/Notebooks/images/lynch_ca.png)

<div class="alert alert-info">
<h3> Your turn</h3>
 <p>Look at the URL. What happens if you change the word 'lynching' to 'murder'? 

What happens to the URL when you go to the second page? Can you get to page 251?

</div>

<details>
<summary>Sample answer url</summary> 
<code style="background-color: white">
https://chroniclingamerica.loc.gov/search/pages/results/?andtext=murder&page=251&sort=relevance
</code>
</details>

What if we append ``&format=json`` to the end of the search URL? 


http://chroniclingamerica.loc.gov/search/pages/results/?andtext=lynching&format=json


[``requests``](http://docs.python-requests.org/en/master/) is a useful and commonly used HTTP library for Python. It is not a part of the default installation, but is included with Anaconda Python Distribution. 

In [1]:
import requests

It would be possible to use the API URL and parameters directly in the requests command, but since the most likely scenario involves making repeating calls to ``requests`` as part of a loop -- the search returned less than 1% of the results -- I store the strings first. 

In [2]:
base_url   = 'http://chroniclingamerica.loc.gov/search/pages/results/'
parameters = '?andtext=lynching&format=json'

url = base_url + parameters
print(url)

http://chroniclingamerica.loc.gov/search/pages/results/?andtext=lynching&format=json


`requests.get()` is used for both accessing websites and APIs. The command can be modified with several arguements, but at a minimum, it requires the URL.

In [3]:
r = requests.get(base_url + parameters)

`r` is a `requests` response object. Any JSON returned by the server will automatically be stored in `.json()` format.

In [4]:
r.json()

{'totalItems': 1006491,
 'endIndex': 20,
 'startIndex': 1,
 'itemsPerPage': 20,
 'items': [{'sequence': 1,
   'county': ['Wayne'],
   'edition': None,
   'frequency': 'Weekly',
   'id': '/lccn/2013218776/1921-10-22/ed-1/seq-1/',
   'subject': ['Dearborn (Mich.)--Newspapers.',
    'Michigan--Dearborn.--fast--(OCoLC)fst01205237',
    'Michigan--Periodicals.',
    'Michigan--Wayne County.--fast--(OCoLC)fst01206628',
    'Michigan.--fast--(OCoLC)fst01208387',
    'Wayne County (Mich.)--Newspapers.'],
   'city': ['Dearborn'],
   'date': '19211022',
   'title': 'Dearborn independent. [volume]',
   'end_year': 1927,
   'note': ['"The Ford international weekly" appears with title in masthead, <Jan 1919-Apr. 1925>.',
    'Archived issues are available in digital format from the Library of Congress Chronicling America online collection.',
    'Available on microfilm from the Auburn University Libraries.',
    'Description based on: Vol. 7, no. 27 (Jan 31, 1908).',
    'Includes editorial page by

In [5]:
search_json = r.json()

JSONs are dictionary like objects, in that they have keys (think variable names) and values. `.keys()` returns a list of the keys.

In [6]:
search_json.keys()

dict_keys(['totalItems', 'endIndex', 'startIndex', 'itemsPerPage', 'items'])

You can return the value of any key by putting the key name in brackets.

In [7]:
search_json['totalItems']

1006491

<div class="alert alert-info">
<h3> Your turn</h3>
 <p>What else is in there? Where is the stuff we want?
</div>

<details>
<summary>Sample answer code</summary> 
<code style="background-color: white">
search_json['items']
</code>
</details>

As is often the case with results from an API, most of the keys and values are metadata about either the search or what is being returned. These let you know if the search is returning what you want, which is particularly important when you are making multiple calls to the API. 

The data I'm intereted in is all in `items`. 

In [ ]:
type(search_json['items'])

In [ ]:
len(search_json['items'])

`items` is a list with 20 items.

In [ ]:
type(search_json['items'][3])

Each of the 20 items in the list is a dictionary. 

In [ ]:
first_item = search_json['items'][0]

first_item.keys()

<div class="alert alert-info">
<h3> Your turn</h3>
 <p>What is the title of the first item?
</div>

<details>
<summary>Sample answer code</summary> 
<code style="background-color: white">
print(first_item['title'])
</code>
</details>

While a standard CSV file has a header row that describes the contents of each column, a JSON file has keys identifying the values found in each case. Importantly, these keys need not be the same for each item. Additionally, values don't have to be numbers of strings, but could be lists or dictionaries. For example, this JSON could have included a `newspaper` key that was a dictionary with all the metadata about the newspaper where the article and issue was published, an `article` key that include the article specific information as another dictionary, and a `text` key whose value was a string with the article text.

As before, we can examine the contents of a particular item, such as the publication's `title`.

In [ ]:
first_item['ocr_eng']

`print` will make things look a little cleaner, converting, for example, `\n` to a line break.

In [ ]:
print(first_item['ocr_eng'])

In [ ]:
print(first_item['ocr_eng'][:200])

The easiest way to view or analyze this data is to convert it to a dataset-like structure. While Python does not have a built-in dataframe type, the popular `pandas` library does. By convention, it is imported as `pd`.

In [ ]:
import pandas as pd

# Make sure all columns are displayed
pd.set_option("display.max_columns",101)

`pandas` is pretty smart about importing different JSON-type objects and converting them to dataframes with its `.DataFrame()` function.

In [ ]:
df = pd.DataFrame(search_json['items'])

df.head(6)

Note that I converted `search_json['items']` to a dataframe and not the entire JSON file. This is because I wanted each row to be an article. 

If this dataframe contained all the items you were looking for, it would be easy to save this to a csv file for storage and later analysis.

In [ ]:
df.to_csv('lynching_articles.csv', index = False)

On a Mac, you can use the 

In [ ]:
!head slavery_articles.csv

In [ ]:
df.to_json('slavery_articles.json', orient='records')

In [ ]:
!head slavery_articles.json

<div class="alert alert-info">
<h3> Your Turn</h3>
<p> Conduct your own search of the API. Store the results in a dataframe.

</div>




<details>
<summary>Sample answer code</summary> 
<code style="background-color: white">
search_word = 'robbery'


base_url   = 'http://chroniclingamerica.loc.gov/search/pages/results/'
parameters = '?andtext=' + search_word + '&format=json'

r = requests.get(base_url + parameters)
search_json = r.json()
df = pd.DataFrame(search_json['items'])

</code>
</details>

### A different API

![](https://github.com/nealcaren/ScrapingData/raw/master/Notebooks/images/fbi_most_wanted.png)

In [ ]:
r = requests.get('https://api.fbi.gov/wanted/v1/list?pageSize=20&page=1&sort_on=modified')

In [ ]:
r.json()

In [ ]:
r.json().keys()

In [ ]:
r.json()['total']

In [ ]:
df = pd.DataFrame(r.json()['items'])
df

<div class="alert alert-info">
<h3> Your turn</h3>
<p> Conduct your own search of the API. Change the page size to 200. Store the results in a csv file.

</div>




<details>
<summary>Sample answer code</summary> 
<code style="background-color: white">
r = requests.get('https://api.fbi.gov/wanted/v1/list?pageSize=200&page=1&sort_on=modified')
mw_df = pd.DataFrame(r.json()['items'])
mw_df.to_csv('mw.csv')

</code>
</details>


In [ ]:
base_url = 'https://api.fbi.gov/wanted/v1/list'

In [ ]:
parameters = {'pageSize' : 20}

In [ ]:
r = requests.get(base_url, 
                 params = parameters)

In [ ]:
r.url

In [ ]:
pd.DataFrame(r.json()['items'])

<div class="alert alert-info">
<h3> Your turn</h3>
<p>Set a page parameter to "2" to get the second page of results.
</div>

<details>
<summary>Sample answer code</summary> 
<code style="background-color: white">
base_url = 'https://api.fbi.gov/wanted/v1/list'
parameters = {'pageSize' : 20,
              'page'     : 2}

r = requests.get(base_url, 
                 params = parameters)
pd.DataFrame(r.json()['items'])
</code>
</details>

## A third API
![](https://github.com/nealcaren/ScrapingData/raw/master/Notebooks/images/chicago_crime.png)

[link](https://dev.socrata.com/foundry/data.cityofchicago.org/ijzp-q8t2)

<div class="alert alert-info">
<h3>In a group </h3>
<p>Read about the API on the <a href='https://dev.socrata.com/foundry/data.cityofchicago.org/ijzp-q8t2'>website</a>. Then create a dataframe that contains that take place in a residence. Run some descriptive statistics on your data.
</div>


<details>
<summary>Sample answer code</summary> 
<code style="background-color: white">
base_url = 'https://data.cityofchicago.org/resource/ijzp-q8t2.json'

parameters = {'location_description' : 'RESIDENCE',
              }

r = requests.get(base_url, 
                 params = parameters)
df = pd.DataFrame(r.json())

df['primary_type'].value_counts()
df['arrest'].value_counts()
pd.crosstab(df['primary_type'] ,df['arrest'], normalize='index')
</code>
</details>

